In [ ]:
import numpy as np # creating a short cut to acces numpy faster in future code
import matplotlib.pyplot as plt #importing matplotlib tool as a shortcut to be able to plot my conclusions. 
from numpy import loadtxt #importing a usefull tool from numpy

def read(file): #defining a new function to read in external files
    data = loadtxt(file) #calling the extracted file data
    data_new = data.transpose() # transposing the values, so I only got 3 rows of data and can acces it easier
    time = data_new[0] #the first row in this transposed data is the time the observations were made 
    flux = data_new[1] #the second row is the flux of the star
    error = data_new[2] #the last row is the error for the y-values(flux)
    
    return time, flux, error #returning three lists containing the unpacked values





time, flux, error = read("Transit_model_LC_data.txt") #taking the values outside of my function to make them globally accesibble 
#code is from 2021 project

In [ ]:
import batman                         #importing the batman module to help plotting the transit curve


params = batman.TransitParams()       #object to store transit parameters
params.t0 = 3.75                        #time of inferior conjunction
params.per = 4.78                        #orbital period
params.rp = 0.09                       #planet radius (in units of stellar radii)
params.a = 10.                        #semi-major axis (in units of stellar radii)
params.inc = 87.                      #orbital inclination (in degrees)
params.ecc = 0.                       #eccentricity
params.w = 0.                        #longitude of periastron (in degrees)
params.limb_dark = "nonlinear"        #limb darkening model
params.u = [0.5, 0.1, 0.1, -0.1]      #limb darkening coefficients [u1, u2, u3, u4]


m = batman.TransitModel(params, time)    #initializes model
flux_bat = m.light_curve(params)          #calculates light curve


In [ ]:
fig = plt.figure(figsize=(20, 20)) # increasing the plot size to better distinguish between the values
plt.errorbar(time, flux, fmt='.', yerr=error) #plotting the errorbar
plt.plot(time, flux,"o") #plotting the time for the x-axis and the radial velocity for the y-axis
plt.plot(time, flux_bat)
plt.title(f"Exoplanet", fontsize=40)
plt.xlabel('Time in Days', fontsize=40)
plt.ylabel('Flux ', fontsize=40)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.show()

In [ ]:
from scipy.optimize import minimize #importing important modules
import math


In [ ]:
m = batman.TransitModel(params, time)    #initializes model
flux_bat = m.light_curve(params)

def fun(x0,a , b, c):#defining a function that returns a gaussian value to be minimised
   
    params.t0 = x0[0]                        #time of inferior conjunction
    params.per = x0[1]                       #orbital period
    params.rp = x0[2]                       #planet radius (in units of stellar radii)
    params.a = x0[3]                       #semi-major axis (in units of stellar radii)
    params.inc = x0[4]                      #orbital inclination (in degrees)
    params.ecc = 0.                       #eccentricity
    params.w = 0.                       #longitude of periastron (in degrees)
    params.limb_dark = "nonlinear"        #limb darkening model
    params.u = [0.5,0.1,0.1,-0.1]
    
    
    
    
    flux_bat = m.light_curve(params)
    if 0. < x0[0]  and 0.0 < x0[1] and 0.0 < x0[2] and 0.0 < x0[3] and 0.0 < x0[4] < 90  : #setting physical boundaries for some parameters
        return sum(((b-flux_bat)/c)**2)
    else:
         return float('inf')

    
         

    
                       

In [ ]:
x0 = [3.75,4.78,0.09,10.,87.] #using my estimated parameters as starting parameters

res = minimize(fun, x0, args = (time, flux, error), method='Nelder-Mead', tol=1e-6 ) #running the minimize function
x = res.x.tolist() #saving the minimised values as a list
true = res.x #saving the minimised values as an array

In [ ]:
params = batman.TransitParams()       #object to store transit parameters
params.t0 = x[0]                        #time of inferior conjunction
params.per = x[1]                       #orbital period
params.rp = x[2]                       #planet radius (in units of stellar radii)
params.a = x[3]                       #semi-major axis (in units of stellar radii)
params.inc = x[4]                      #orbital inclination (in degrees)
params.ecc = 0.                       #eccentricity
params.w = 0.                       #longitude of periastron (in degrees)
params.limb_dark = "nonlinear"        #limb darkening model
params.u = [0.5,0.1,0.1,-0.1]
flux_bat1 = m.light_curve(params)

In [ ]:

fig = plt.figure(figsize=(10, 10)) # increasing the plot size to better distinguish between the values
plt.errorbar(time, flux, fmt='.', yerr=error) #plotting the errorbar
plt.plot(time, flux,"o") #plotting the time for the x-axis and the radial velocity for the y-axis
plt.plot(time, flux_bat1)
plt.title(f"Exoplanet", fontsize=40)
plt.xlabel('Time in Days', fontsize=20)
plt.ylabel('Flux ', fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show() #plotting with my minimised values

In [ ]:
phase= np.mod(time-x[0],x[1])/x[1] # creating a phased plot for all the values
phase[phase>0.5 ]-=1

m = batman.TransitModel(params, phase)
plt.plot(phase, flux,"o")
plt.errorbar(phase, flux,  fmt='o', yerr=error)

plt.plot(phase, flux_bat, '-.')
    



In [ ]:
m = batman.TransitModel(params, time)
theta = x #naming my minimised values from minimize theta
def log_likelihood(theta, time_fun, flux_fun, error_fun): # creating a log likelihood function for my emcee module
    
    
    params = batman.TransitParams()       #object to store transit parameters
    params.t0 = theta[0]                        #time of inferior conjunction
    params.per = theta[1]                 #orbital period
    params.rp = theta[2]                      #planet radius (in units of stellar radii)
    params.a = theta[3]                        #semi-major axis (in units of stellar radii)
    params.inc = theta[4]                      #orbital inclination (in degrees)
    params.ecc = 0.                       #eccentricity
    params.w = 0.                       #longitude of periastron (in degrees)
    params.limb_dark = "nonlinear"        #limb darkening model
    params.u = [0.5,0.1,0.1,-0.1]  
    model = m.light_curve(params)
    
    
   
    return -0.5 * np.sum(((flux_fun - model) / error_fun)**2)


In [ ]:
def log_prior(theta): #creating a log prior that checks if the values are in the physical boundaries
    
    if 0. < theta[0] <10 and 0.0 < theta[1] < 10 and 0.0 < theta[2] < 1 and 0.0 < theta[3] < 50 and 0.0 < theta[4] < 90  :
        return 0.0
  
        
    return -np.inf
    

In [ ]:

def log_probability(theta, time_fun, flux_fun, error_fun): #creating a log probability function
    lp = log_prior(theta)
    
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, time_fun, flux_fun, error_fun)
    

In [ ]:

import emcee # importing the emcee module
ndim = 5 # creating a dimension for every value I need to fit
nwalkers = 50 #creating a number of walkers for the MCMC
pos = true + 1e-4 * np.random.randn(50, 5) #creating a semi-random starting point for the walkers
nwalkers, ndim = pos.shape
sampler = emcee.EnsembleSampler(
    nwalkers, ndim, log_probability, args=( time, flux, error),
moves=emcee.moves.StretchMove()) #creating a sampler for my MCMC 
sampler.run_mcmc(pos , 5000, progress=True); #letting my emcee run 5000 times

In [ ]:
labels = ["Time(0)","Orbital" "Period","Planet"" Radius", "Semi-Major"" Axis" ,"Orbital"" Inclination"]#creating a list of labels for my plot
fig, axes = plt.subplots(5, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()

for i in range(ndim): #creating five plots for the MCMC values
    ax = axes[i]
    

    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])

In [ ]:
labels = ["Time" "" "of" "Inferior" "Conjunction","Orbital" "Period","Planet"" Radius", "Semi-Major"" Axis" ,"Orbital"" Inclination"]
import corner

flat_samples = sampler.get_chain( flat=True) #creating corner plots for the results of the MCMC with the blue line being the value calculated by minimize
fig = corner.corner(
    flat_samples,   truths= true, labels = labels
)

In [ ]:
from IPython.display import display, Math
results1 = [] #creating a list to store the emcee value of my result
result1_uncertainty = []
for i in range(ndim): #printing the mean values and the corresponding one-sigma value
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{{2:.3f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labels[i])
    display(Math(txt))
    result1_uncertainty.append([ mcmc[1], q[0], q[1]])
    results1.append(mcmc[1])


In [ ]:
t = np.linspace(time[0], time[-1], 2000) # plotting the lightcurve from the emcee values over my data
params = batman.TransitParams()       #object to store transit parameters
params.t0 = results1[0]                     #time of inferior conjunction
params.per = results1[1]                      #orbital period
params.rp = results1[2]                       #planet radius (in units of stellar radii)
params.a = results1[3]                        #semi-major axis (in units of stellar radii)
params.inc = results1[4]                      #orbital inclination (in degrees)
params.ecc = 0.                       #eccentricity
params.w = 0.                       #longitude of periastron (in degrees)
params.limb_dark = "nonlinear"        #limb darkening model
params.u = [0.5,0.1,0.1,-0.1] 
m = batman.TransitModel(params, t)    #initializes model
flux_bat = m.light_curve(params)

fig = plt.figure(figsize=(10, 10))
plt.plot(time, flux,"o")
plt.errorbar(time, flux, fmt='.', yerr=error) #plotting the errorbar
plt.plot(t, flux_bat)
plt.title(f"Exoplanet", fontsize=40)
plt.xlabel('Time in Days', fontsize=20)
plt.ylabel('Flux ', fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show() # plotting the data and the light curve with the parameter values from emcee